# Hyperparameter Tuning For Music Generation
## Import Packages

In [114]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  
import pandas as pd
import time
from sklearn.metrics import accuracy_score

## Load and Prepare Data

In [115]:
df = pd.read_csv("bigMusicInfo.csv")
df

,Unnamed: 0,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,spectral_flux_mean,spectral_rolloff_mean,mfcc_1_mean,...,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std,Row_Index,Genre,GenreID
0,0,0.126249,0.096276,0.092643,0.108992,0.087193,0.075386,0.082652,0.096276,0.089918,...,0.062670,0.063579,0.066303,0.081744,0.096276,0.099909,0.088102,0.0,blues,0
1,1,0.029973,0.019982,0.010899,0.017257,0.019074,0.010899,0.014532,0.021798,0.015441,...,0.051771,0.043597,0.049955,0.069936,0.069936,0.055404,0.053588,1.0,blues,1
2,2,0.089918,0.079927,0.082652,0.079927,0.066303,0.065395,0.069028,0.075386,0.074478,...,0.059946,0.051771,0.044505,0.044505,0.050863,0.051771,0.042688,2.0,blues,2
3,3,0.019982,0.020890,0.020890,0.016349,0.012716,0.011807,0.008174,0.006358,0.008174,...,0.034514,0.036331,0.029973,0.022707,0.025431,0.034514,0.030881,3.0,blues,3
4,4,0.080836,0.098093,0.099909,0.100817,0.099001,0.080836,0.060854,0.057221,0.063579,...,0.181653,0.219800,0.259764,0.290645,0.281562,0.296094,0.324251,4.0,blues,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,994,0.069028,0.070845,0.069028,0.074478,0.080836,0.075386,0.074478,0.065395,0.069936,...,0.076294,0.079019,0.078111,0.083560,0.087193,0.066303,0.066303,994.0,rock,95
995,995,0.114441,0.107175,0.099909,0.111717,0.125341,0.111717,0.105359,0.108084,0.095368,...,0.117166,0.099001,0.092643,0.098093,0.104450,0.180745,0.219800,995.0,rock,96
996,996,0.087193,0.073569,0.061762,0.055404,0.064487,0.080836,0.078111,0.066303,0.072661,...,0.065395,0.063579,0.077203,0.079019,0.081744,0.087193,0.092643,996.0,rock,97
997,997,0.016349,0.016349,0.020890,0.022707,0.019982,0.019074,0.018165,0.017257,0.019074,...,0.039964,0.041780,0.032698,0.029064,0.032698,0.040872,0.039055,997.0,rock,98


In [116]:
df_train = (df.copy()).iloc[:, 1:-3]

In [117]:
df_train.head()

,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,spectral_flux_mean,spectral_rolloff_mean,mfcc_1_mean,mfcc_2_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,0.126249,0.096276,0.092643,0.108992,0.087193,0.075386,0.082652,0.096276,0.089918,0.089010,...,0.069936,0.073569,0.065395,0.062670,0.063579,0.066303,0.081744,0.096276,0.099909,0.088102
1,0.029973,0.019982,0.010899,0.017257,0.019074,0.010899,0.014532,0.021798,0.015441,0.011807,...,0.092643,0.099001,0.075386,0.051771,0.043597,0.049955,0.069936,0.069936,0.055404,0.053588
2,0.089918,0.079927,0.082652,0.079927,0.066303,0.065395,0.069028,0.075386,0.074478,0.067212,...,0.092643,0.079019,0.066303,0.059946,0.051771,0.044505,0.044505,0.050863,0.051771,0.042688
3,0.019982,0.020890,0.020890,0.016349,0.012716,0.011807,0.008174,0.006358,0.008174,0.010899,...,0.039964,0.048138,0.034514,0.034514,0.036331,0.029973,0.022707,0.025431,0.034514,0.030881
4,0.080836,0.098093,0.099909,0.100817,0.099001,0.080836,0.060854,0.057221,0.063579,0.055404,...,0.119891,0.116258,0.120799,0.181653,0.219800,0.259764,0.290645,0.281562,0.296094,0.324251


In [118]:
X = df_train.values / 16
y = df.values[:, -2]

In [119]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y)

In [120]:
X_train.shape

(699, 136)

In [121]:
y_train.shape

(699,)

In [122]:
np.unique(y_train)

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

In [123]:
#(np.bincount(y_train) / len(y_train)).round(3)

In [124]:
X_train_orig = df_train.values[:, :-1]

In [125]:
np.unique(y_train)

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

In [126]:
y_train[:50].reshape(5, 10)

array([['hiphop', 'classical', 'rock', 'hiphop', 'rock', 'hiphop',
        'disco', 'metal', 'country', 'country'],
       ['metal', 'country', 'blues', 'pop', 'classical', 'jazz', 'jazz',
        'rock', 'reggae', 'disco'],
       ['hiphop', 'reggae', 'classical', 'hiphop', 'jazz', 'blues',
        'classical', 'country', 'jazz', 'hiphop'],
       ['metal', 'reggae', 'pop', 'country', 'metal', 'rock', 'disco',
        'classical', 'pop', 'rock'],
       ['blues', 'disco', 'disco', 'blues', 'metal', 'country',
        'classical', 'disco', 'reggae', 'country']], dtype=object)

In [127]:
#df_test = pd.read_csv('optdigits.tes.csv', header=None)

In [128]:
#X_test = df_test.values[:, :-1] / 16
#y_test = df_test.values[:, -1]

In [129]:
X_test.shape

(300, 136)

In [130]:
y_test.shape

(300,)

## Decision Tree

In [131]:
from sklearn.tree import DecisionTreeClassifier

### Finding Best Parameter

In [132]:
depth_range = np.arange(5, 31)
criterion_range = ['gini', 'entropy']
dt_grid = {'max_depth': depth_range, 'criterion': criterion_range}
dt_grid

{'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30]),
 'criterion': ['gini', 'entropy']}

In [133]:
from sklearn.model_selection import GridSearchCV

In [134]:
dt_gs = GridSearchCV(estimator=DecisionTreeClassifier(), 
                  param_grid=dt_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [135]:
current_time = time.time()
dt_gs.fit(X_train, y_train)
dt_gs_fit_time = time.time() - current_time

In [136]:
dt_gs.best_params_

{'criterion': 'gini', 'max_depth': 10}

In [137]:
dt_gs.best_score_

0.20886947584789312

In [138]:
best_dt = dt_gs.best_estimator_
best_dt

DecisionTreeClassifier(max_depth=10)

In [139]:
current_time = time.time()
y_train_predict_dt_gs = dt_gs.predict(X_train)
dt_gs_train_time = time.time() - current_time
dt_gs_train = accuracy_score(y_train, y_train_predict_dt_gs)
dt_gs_train

0.6466380543633763

In [140]:
current_time = time.time()
y_test_predict_dt_gs = dt_gs.predict(X_test)
dt_gs_test_time = time.time() - current_time
dt_gs_test = accuracy_score(y_test, y_test_predict_dt_gs)
dt_gs_test

0.19666666666666666

### Fitting, Training, Testing with Best Params

In [141]:
stump = DecisionTreeClassifier(criterion= 'gini', max_depth=dt_gs.best_params_["max_depth"])

In [142]:
current_time = time.time()
stump.fit(X_train, y_train)
stump_fit_time = time.time() - current_time

In [143]:
current_time = time.time()
y_train_predict_stump = stump.predict(X_train)
stump_train_time = time.time() - current_time
stump_train = accuracy_score(y_train, y_train_predict_stump)
stump_train

0.6452074391988555

In [144]:
current_time = time.time()
y_test_predict_stump = stump.predict(X_test)
stump_test_time = time.time() - current_time
stump_test = accuracy_score(y_test, y_test_predict_stump)
stump_test

0.18333333333333332

# Random Forest

In [145]:
from sklearn.ensemble import RandomForestClassifier

In [146]:
depth_range = np.arange(5, 31)
n_trees_range = np.arange(30, 210, 10)
rf_grid = {'max_depth': depth_range, 'n_estimators': n_trees_range}
rf_grid

{'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30]),
 'n_estimators': array([ 30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130, 140, 150,
        160, 170, 180, 190, 200])}

In [147]:
rf_gs = GridSearchCV(estimator=RandomForestClassifier(), 
                  param_grid=rf_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [148]:
current_time = time.time()
rf_gs.fit(X_train, y_train)
rf_gs_fit_time = time.time() - current_time

In [149]:
rf_gs.best_params_

{'max_depth': 22, 'n_estimators': 90}

In [150]:
rf_gs.best_score_

0.3033299075025694

In [151]:
best_rf = rf_gs.best_estimator_
best_rf

RandomForestClassifier(max_depth=22, n_estimators=90)

In [152]:
current_time = time.time()
y_train_predict_rf_gs = rf_gs.predict(X_train)
rf_gs_train_time = time.time() - current_time
rf_gs_train = accuracy_score(y_train, y_train_predict_rf_gs)
rf_gs_train

0.9985693848354793

In [153]:
current_time = time.time()
y_test_predict_rf_gs = rf_gs.predict(X_test)
rf_gs_test_time = time.time() - current_time
rf_gs_test = accuracy_score(y_test, y_test_predict_rf_gs)
rf_gs_test

0.30666666666666664

### Fitting, Training, Testing with Best Params

In [154]:
rf = RandomForestClassifier(criterion= 'gini', n_estimators=rf_gs.best_params_['n_estimators'], max_depth=rf_gs.best_params_['max_depth'])

In [155]:
current_time = time.time()
rf.fit(X_train, y_train)
rf_fit_time = time.time() - current_time

In [156]:
current_time = time.time()
y_train_pred_rf = rf.predict(X_train)
rf_train_time = time.time() - current_time
RF_train = accuracy_score(y_train, y_train_pred_rf)
RF_train

0.9985693848354793

In [157]:
current_time = time.time()
y_test_pred_rf = rf.predict(X_test)
rf_test_time = time.time() - current_time
RF_test = accuracy_score(y_test, y_test_pred_rf)
RF_test

0.3

# Logistic Regression

In [158]:
# I dont know what parameters to tune

# Linear SVM

In [159]:
from sklearn.svm import LinearSVC

### Finding best params

In [160]:
c_range = np.arange(5.0, 31.0)
lSVM_grid = {'C': c_range}
lSVM_grid

{'C': array([ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,
        18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30.])}

In [161]:
lSVM_gs = GridSearchCV(estimator=LinearSVC(max_iter=250), 
                  param_grid=lSVM_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [162]:
current_time = time.time()
lSVM_gs.fit(X_train, y_train)
lSVM_gs_fit_time = time.time() - current_time

D:\MiniConda\envs\ai_env\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [163]:
lSVM_gs.best_params_

{'C': 10.0}

In [164]:
current_time = time.time()
y_pred_train_lSVM_gs = lSVM_gs.predict(X_train)
lSVM_gs_train_time = time.time() - current_time
lSVM_gs_train = accuracy_score(y_train, y_pred_train_lSVM_gs)
lSVM_gs_train

0.32045779685264664

In [165]:
current_time = time.time()
y_pred_test_lSVM_gs = lSVM_gs.predict(X_test)
lSVM_gs_test_time = time.time() - current_time
lSVM_gs_test = accuracy_score(y_test, y_pred_test_lSVM_gs)
lSVM_gs_test

0.20666666666666667

### Fitting, Training, Testing with Best Params

In [166]:
clf = LinearSVC(max_iter = 250, C = lSVM_gs.best_params_['C'])

In [167]:
current_time = time.time()
clf.fit(X_train, y_train)
clf_fit_time = time.time() - current_time

D:\MiniConda\envs\ai_env\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [168]:
current_time = time.time()
y_pred_train_svm_lin = clf.predict(X_train)
clf_train_time = time.time() - current_time
LinearSVM_train = accuracy_score(y_train, y_pred_train_svm_lin)
LinearSVM_train

0.3190271816881259

In [169]:
current_time = time.time()
y_pred_test_svm_lin = clf.predict(X_test)
clf_test_time = time.time() - current_time
LinearSVM_test = accuracy_score(y_test, y_pred_test_svm_lin)
LinearSVM_test

0.20666666666666667

# Gaussian SVM

In [170]:
from sklearn.svm import SVC

### Finding Best Params

In [171]:
c_range = np.arange(5.0, 31.0)
gamma_range = np.logspace(-9, 3, 13)
gaus_grid = {'C': c_range, 'gamma': gamma_range}
gaus_grid

{'C': array([ 5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,
        18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30.]),
 'gamma': array([1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02,
        1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])}

In [172]:
gaus_gs = GridSearchCV(estimator= SVC(max_iter=250, kernel = 'rbf'), 
                  param_grid=gaus_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [173]:
current_time = time.time()
gaus_gs.fit(X_train, y_train)
gaus_gs_fit_time = time.time() - current_time

D:\MiniConda\envs\ai_env\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [174]:
gaus_gs.best_params_

{'C': 10.0, 'gamma': 1000.0}

In [175]:
current_time = time.time()
y_pred_train_gaus_gs = gaus_gs.predict(X_train)
gaus_gs_train_time = time.time() - current_time
gaus_gs_train = accuracy_score(y_train, y_pred_train_gaus_gs)
gaus_gs_train

0.9799713876967096

In [176]:
current_time = time.time()
y_pred_test_gaus_gs = gaus_gs.predict(X_test)
gaus_gs_test_time = time.time() - current_time
gaus_gs_test = accuracy_score(y_test, y_pred_test_gaus_gs)
gaus_gs_test

0.35

### Fitting, Training, Testing with Best Params

In [177]:
svm_gaussian = SVC(kernel='rbf', max_iter = 250, C= gaus_gs.best_params_['C'], gamma = gaus_gs.best_params_['gamma'])

In [178]:
current_time = time.time()
svm_gaussian.fit(X_train, y_train)
gaus_fit_time = time.time() - current_time

D:\MiniConda\envs\ai_env\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [179]:
current_time = time.time()
y_pred_gaussian_svm = svm_gaussian.predict(X_train)
gaus_train_time = time.time() - current_time
GaussianSVM_train = accuracy_score(y_train, y_pred_gaussian_svm)
GaussianSVM_train

0.9799713876967096

In [180]:
current_time = time.time()
y_test_gaussian_svm = svm_gaussian.predict(X_test)
gaus_test_time = time.time() - current_time
GaussianSVM_test = accuracy_score(y_test, y_test_gaussian_svm)
GaussianSVM_test

0.35

# K Nearest Neighbors

In [181]:
from sklearn.neighbors import KNeighborsClassifier

### Finding Best Params

In [182]:
n_neighbors_range = np.arange(1, 15)
weights_range = ['uniform', 'distance']
knn_grid = {'n_neighbors': n_neighbors_range, 'weights': weights_range}
knn_grid

{'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 'weights': ['uniform', 'distance']}

In [183]:
knn_gs = GridSearchCV(estimator= KNeighborsClassifier(), 
                  param_grid=knn_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [184]:
current_time = time.time()
knn_gs.fit(X_train, y_train)
knn_gs_fit_time = time.time() - current_time

In [185]:
knn_gs.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [186]:
current_time = time.time()
y_pred_train_knn_gs = knn_gs.predict(X_train)
knn_gs_train_time = time.time() - current_time
knn_gs_train = accuracy_score(y_train, y_pred_train_knn_gs)
knn_gs_train

0.9985693848354793

In [187]:
current_time = time.time()
y_pred_test_knn_gs = knn_gs.predict(X_test)
knn_gs_test_time = time.time() - current_time
knn_gs_test = accuracy_score(y_test, y_pred_test_knn_gs)
knn_gs_test

0.22

### Fitting, Training, Testing with Best Params

In [188]:
knn = KNeighborsClassifier(n_neighbors= knn_gs.best_params_['n_neighbors'], weights= knn_gs.best_params_['weights'])

In [189]:
current_time = time.time()
knn.fit(X_train, y_train)
KNN_fit_time = time.time() - current_time

In [190]:
current_time = time.time()
y_pred_train_knn = knn.predict(X_train)
KNN_train_time = time.time() - current_time
KNearest_train = accuracy_score(y_train, y_pred_train_knn)
KNearest_train

0.9985693848354793

In [191]:
current_time = time.time()
y_pred_test_knn = knn.predict(X_test)
KNN_test_time = time.time() - current_time
KNearest_test = accuracy_score(y_test, y_pred_test_knn)
KNearest_test

0.22

# Adaboost

In [192]:
from sklearn.ensemble import AdaBoostClassifier

### Finding Best Params

In [193]:
n_estimators_range = np.arange(40,60)
#max_depth_range = np.arange(1,7)
ada_grid = {'n_estimators': n_estimators_range}
ada_grid
#, 'max_depth': max_depth_range

{'n_estimators': array([40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
        57, 58, 59])}

In [194]:
max_depth_range = np.arange(1,7)
dt_ada_grid = {'max_depth': max_depth_range}
dt_ada_grid
#, 'max_depth': max_depth_range

{'max_depth': array([1, 2, 3, 4, 5, 6])}

In [195]:
model_est = GridSearchCV(estimator= DecisionTreeClassifier(), 
                  param_grid=dt_ada_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [196]:
model_est.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6])},
             scoring='accuracy')

In [197]:
model_est.best_params_

{'max_depth': 6}

In [198]:
ada_gs = GridSearchCV(estimator= AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = model_est.best_params_['max_depth'])), 
                  param_grid=ada_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=5,
                  n_jobs=-1)

In [199]:
current_time = time.time()
ada_gs.fit(X_train, y_train)
ada_gs_fit_time = time.time() - current_time

In [200]:
ada_gs.best_params_

{'n_estimators': 56}

In [201]:
current_time = time.time()
y_pred_train_ada_gs = ada_gs.predict(X_train)
ada_gs_train_time = time.time() - current_time
ada_gs_train = accuracy_score(y_train, y_pred_train_ada_gs)
ada_gs_train

0.9899856938483548

In [202]:
current_time = time.time()
y_pred_test_ada_gs = ada_gs.predict(X_test)
ada_gs_test_time = time.time() - current_time
ada_gs_test = accuracy_score(y_test, y_pred_test_ada_gs)
ada_gs_test

0.27666666666666667

### Fitting, Training, Testing with Best Params

In [203]:
adaB = AdaBoostClassifier(DecisionTreeClassifier(max_depth = model_est.best_params_['max_depth']), n_estimators = ada_gs.best_params_['n_estimators'])

In [204]:
current_time = time.time()
adaB.fit(X_train, y_train)
adaB_fit_time = time.time() - current_time

In [205]:
current_time = time.time()
y_pred_train_adaB = adaB.predict(X_train)
adaB_train_time = time.time() - current_time
adaB_train = accuracy_score(y_train, y_pred_train_adaB)
adaB_train

0.9928469241773963

In [206]:
current_time = time.time()
y_pred_test_adaB = adaB.predict(X_test)
adaB_test_time = time.time() - current_time
adaB_test = accuracy_score(y_test, y_pred_test_adaB)
adaB_test

0.29

# Results

### Accuracy of each model while finding the best parameter:

In [207]:
print("The accuracy of Decision Tree for the training during hyperameter tuning:  " , dt_gs_train)
print("The accuracy of Decision Tree for the test during hyperameter tuning:  " , dt_gs_test, "\n")

print("The accuracy of Linear SVM for the training during hyperameter tuning:  " , lSVM_gs_train)
print("The accuracy of Linear SVM for the test during hyperameter tuning:  " , lSVM_gs_test, "\n")

print("The accuracy of Gaussian SVM for the training during hyperameter tuning:  " , gaus_gs_train)
print("The accuracy of Gaussian SVM for the test during hyperameter tuning:  " , gaus_gs_train, "\n")

print("The accuracy of K Nearest Neighboors for the training during hyperameter tuning:  " , knn_gs_train)
print("The accuracy of K Nearest Neighboors for the test during hyperameter tuning:  " , knn_gs_test, "\n")

print("The accuracy of Random Forest for the training during hyperameter tuning:  " , rf_gs_train)
print("The accuracy of Random Forest for the test during hyperameter tuning:  " , rf_gs_test, "\n")

print("The accuracy of Adaboost for the training during hyperameter tuning:  " , ada_gs_train)
print("The accuracy of Adaboost for the test during hyperameter tuning:  " , ada_gs_test, "\n")

The accuracy of Decision Tree for the training during hyperameter tuning:   0.6466380543633763
The accuracy of Decision Tree for the test during hyperameter tuning:   0.19666666666666666 

The accuracy of Linear SVM for the training during hyperameter tuning:   0.32045779685264664
The accuracy of Linear SVM for the test during hyperameter tuning:   0.20666666666666667 

The accuracy of Gaussian SVM for the training during hyperameter tuning:   0.9799713876967096
The accuracy of Gaussian SVM for the test during hyperameter tuning:   0.9799713876967096 

The accuracy of K Nearest Neighboors for the training during hyperameter tuning:   0.9985693848354793
The accuracy of K Nearest Neighboors for the test during hyperameter tuning:   0.22 

The accuracy of Random Forest for the training during hyperameter tuning:   0.9985693848354793
The accuracy of Random Forest for the test during hyperameter tuning:   0.30666666666666664 

The accuracy of Adaboost for the training during hyperameter tun

### Time taken for each model during hyperameter tuning:

In [208]:
print("The number of seconds for fitting in Decision Tree during hyperameter tuning:  " , dt_gs_fit_time)
print("The number of seconds for testing the train data in Decision Tree during hyperameter tuning:  " , dt_gs_train_time)
print("The number of seconds for testing the test data in Decision Tree during hyperameter tuning:  " , dt_gs_test_time, "\n")

print("The number of seconds for fitting in Linear SVM during hyperameter tuning:  " , lSVM_gs_fit_time)
print("The number of seconds for testing the train data in Linear SVM during hyperameter tuning:  " , lSVM_gs_train_time)
print("The number of seconds for testing the test data in Linear SVM during hyperameter tuning:  " , lSVM_gs_test_time, "\n")

print("The number of seconds for fitting in Gaussian SVM during hyperameter tuning:  " , gaus_gs_fit_time)
print("The number of seconds for testing the train data in Gaussian SVM during hyperameter tuning:  " , gaus_gs_train_time)
print("The number of seconds for testing the test data in Gaussian SVM during hyperameter tuning:  " , gaus_gs_test_time, "\n")

print("The number of seconds for fitting in K Nearest Neighboors during hyperameter tuning:  " , knn_gs_fit_time)
print("The number of seconds for testing the train data in K Nearest Neighboors during hyperameter tuning:  " , knn_gs_train_time)
print("The number of seconds for testing the test data in K Nearest Neighboors during hyperameter tuning:  " , knn_gs_test_time, "\n")

print("The number of seconds for fitting in Random Forest during hyperameter tuning:  " , rf_gs_fit_time)
print("The number of seconds for testing the train data in Random Forest during hyperameter tuning:  " , rf_gs_train_time)
print("The number of seconds for testing the test data in Random Forest during hyperameter tuning:  " , rf_gs_test_time, "\n")

print("The number of seconds for fitting in Adaboost during hyperameter tuning:  " , ada_gs_fit_time)
print("The number of seconds for testing the train data in Adaboost during hyperameter tuning:  " , ada_gs_train_time)
print("The number of seconds for testing the test data in Adaboost during hyperameter tuning:  " , ada_gs_test_time)

The number of seconds for fitting in Decision Tree during hyperameter tuning:   12.004477977752686
The number of seconds for testing the train data in Decision Tree during hyperameter tuning:   0.0019953250885009766
The number of seconds for testing the test data in Decision Tree during hyperameter tuning:   0.0009980201721191406 

The number of seconds for fitting in Linear SVM during hyperameter tuning:   15.648300647735596
The number of seconds for testing the train data in Linear SVM during hyperameter tuning:   0.0608363151550293
The number of seconds for testing the test data in Linear SVM during hyperameter tuning:   0.0 

The number of seconds for fitting in Gaussian SVM during hyperameter tuning:   23.206920862197876
The number of seconds for testing the train data in Gaussian SVM during hyperameter tuning:   0.11369466781616211
The number of seconds for testing the test data in Gaussian SVM during hyperameter tuning:   0.04388260841369629 

The number of seconds for fitting i

### Accuracy of the models with the best parameters:

In [209]:
print("The accuracy of Decision Tree for the training with the best parameters:  " , stump_train)
print("The accuracy of Decision Tree for the test with the best parameters:  " , stump_test, "\n")

print("The accuracy of Linear SVM for the training with the best parameters:  " , LinearSVM_train)
print("The accuracy of Linear SVM for the test with the best parameters:  " , LinearSVM_test, "\n")

print("The accuracy of Gaussian SVM for the training with the best parameters:  " , GaussianSVM_train)
print("The accuracy of Gaussian SVM for the test with the best parameters:  " , GaussianSVM_train, "\n")

print("The accuracy of K Nearest Neighboors for the training with the best parameters:  " , KNearest_train)
print("The accuracy of K Nearest Neighboors for the test with the best parameters:  " , KNearest_test, "\n")

print("The accuracy of Random Forest for the training with the best parameters:  " , RF_train)
print("The accuracy of Random Forest for the test with the best parameters:  " , RF_test, "\n")

print("The accuracy of Adaboost for the training with the best parameters:  " , adaB_train)
print("The accuracy of Adaboost for the test with the best parameters:  " , adaB_test, "\n")

The accuracy of Decision Tree for the training with the best parameters:   0.6452074391988555
The accuracy of Decision Tree for the test with the best parameters:   0.18333333333333332 

The accuracy of Linear SVM for the training with the best parameters:   0.3190271816881259
The accuracy of Linear SVM for the test with the best parameters:   0.20666666666666667 

The accuracy of Gaussian SVM for the training with the best parameters:   0.9799713876967096
The accuracy of Gaussian SVM for the test with the best parameters:   0.9799713876967096 

The accuracy of K Nearest Neighboors for the training with the best parameters:   0.9985693848354793
The accuracy of K Nearest Neighboors for the test with the best parameters:   0.22 

The accuracy of Random Forest for the training with the best parameters:   0.9985693848354793
The accuracy of Random Forest for the test with the best parameters:   0.3 

The accuracy of Adaboost for the training with the best parameters:   0.9928469241773963
Th

### Time taken for each model with the best parameters

In [210]:
print("The number of seconds for fitting in Decision Tree with the best parameters:  " , stump_fit_time)
print("The number of seconds for testing the train data in Decision Tree with the best parameters:  " , stump_train_time)
print("The number of seconds for testing the test data in Decision Tree with the best parameters:  " , stump_test_time, "\n")

print("The number of seconds for fitting in Linear SVM with the best parameters:  " , clf_fit_time)
print("The number of seconds for testing the train data in Linear SVM with the best parameters:  " , clf_train_time)
print("The number of seconds for testing the test data in Linear SVM with the best parameters:  " , clf_test_time, "\n")

print("The number of seconds for fitting in Gaussian SVM with the best parameters:  " , gaus_fit_time)
print("The number of seconds for testing the train data in Gaussian SVM with the best parameters:  " , gaus_train_time)
print("The number of seconds for testing the test data in Gaussian SVM with the best parameters:  " , gaus_test_time, "\n")

print("The number of seconds for fitting in K Nearest Neighboors with the best parameters:  " , KNN_fit_time)
print("The number of seconds for testing the train data in K Nearest Neighboors with the best parameters:  " , KNN_train_time)
print("The number of seconds for testing the test data in K Nearest Neighboors with the best parameters:  " , KNN_test_time, "\n")

print("The number of seconds for fitting in Random Forest with the best parameters:  " , rf_fit_time)
print("The number of seconds for testing the train data in Random Forest with the best parameters:  " , rf_train_time)
print("The number of seconds for testing the test data in Random Forest with the best parameters:  " , rf_test_time, "\n")

print("The number of seconds for fitting in Adaboost with the best parameters:  " , adaB_fit_time)
print("The number of seconds for testing the train data in Adaboost with the best parameters:  " , adaB_train_time)
print("The number of seconds for testing the test data in Adaboost with the best parameters:  " , adaB_test_time)

The number of seconds for fitting in Decision Tree with the best parameters:   0.0718085765838623
The number of seconds for testing the train data in Decision Tree with the best parameters:   0.0009963512420654297
The number of seconds for testing the test data in Decision Tree with the best parameters:   0.000997304916381836 

The number of seconds for fitting in Linear SVM with the best parameters:   0.48772382736206055
The number of seconds for testing the train data in Linear SVM with the best parameters:   0.0009980201721191406
The number of seconds for testing the test data in Linear SVM with the best parameters:   0.0009980201721191406 

The number of seconds for fitting in Gaussian SVM with the best parameters:   0.08177995681762695
The number of seconds for testing the train data in Gaussian SVM with the best parameters:   0.08976006507873535
The number of seconds for testing the test data in Gaussian SVM with the best parameters:   0.03789663314819336 

The number of seconds 

# Conclusions:

# Observation


# Conclusion


In [214]:
resultsTrain = pd.DataFrame.from_dict({"Adaboost": adaB_train, "Decision Tree" : stump_train, "Linear SVM" : LinearSVM_train, "Gaussian SVM" : GaussianSVM_train, "K Nearest" : KNearest_train, "Random Forest" : RF_train}, orient= 'index')
resultsTrain

,0
Adaboost,0.992847
Decision Tree,0.645207
Linear SVM,0.319027
Gaussian SVM,0.979971
K Nearest,0.998569
Random Forest,0.998569


In [215]:
resultsTest = pd.DataFrame.from_dict({"Adaboost": ada_gs_test, "Decision Tree" : stump_test, "Linear SVM" : LinearSVM_test, "Gaussian SVM" : GaussianSVM_test, "K Nearest" : KNearest_test, "Random Forest" : RF_test}, orient= 'index')
resultsTest

,0
Adaboost,0.276667
Decision Tree,0.183333
Linear SVM,0.206667
Gaussian SVM,0.350000
K Nearest,0.220000
Random Forest,0.300000


In [216]:
resultsTrain.to_csv("C:\\Users\\samue\\Desktop\\AI_Proj\\HyperTuningTrain.csv")
resultsTest.to_csv("C:\\Users\\samue\\Desktop\\AI_Proj\\HyperTuningTest.csv")